# Anonymous Crypto Fintech Company
## Analyst <br>Take Home Python Assignment <br>Ivan Ronceria

### This is a passing solution bank for a python take home assignment given by an anonymous crypto fintech company.


### Question 1

The Grayscale Bitcoin Trust (GBTC) is commonly used as collateral in the institutional crypto
lending market. GBTC was created to track the price of BTC, but it currently trades at a discount to
its Net Asset Value (NAV).

Please write a script using Python to construct a price and discount estimate for GBTC. Please use
the CryptoCompare API (sign up for a free CryptoCompare API key <a href="https://min-api.cryptocompare.com/pricing" target="_blank">here</a>) and the <a href="https://www.cnbc.com/quotes/GBTC" target="_blank">CNBC quote</a>) for
GBTC for this project. You can assume that there is 0.00092683 BTC per share of GBTC.

#### Solution

We begin by importing the required packages for both question 1 and 2.

In [ ]:
from bs4 import BeautifulSoup
import json
import requests
import datetime
import time
import pandas as pd

Next, we define some constants as well.

* **CC_SECRET_KEY** - This is the secret key necessary to access the CryptoCompare API

* **QUOTE_CURRENCY** - For the sake of this assignment, we wish to quote in USD. However, you may change to whatever coin or currency available to the CC API and should still work

* **SYMBOL** - Needed for Q2. Pass any available coin from the CC API

* **MAX_MINS** - This is the maximum dataframe size. This is necessary mostly for Q2. It is set by default to 720 because it is unnecesary to call the maximum data limit (2000), but you can change it to higher value than 720. Changing it lower to 720 will break the code.

* **URL_SINGLE_PRICE** and **URL_MINUTE_OHLCV** - Used for requests.

* **HEADERS** - Used to pass the authentication information to request payload.

In [ ]:
CC_SECRET_KEY = "d3a5b51ce275baa2ee05a6169f7c99fe55abaa9d81ccc6e9803f3a4892ee9ad5"
QUOTE_CURRENCY = "USD"
SYMBOL = input("Please enter the coin you wish to study: ")

MAX_MINS = 720

URL_SINGLE_PRICE = "https://min-api.cryptocompare.com/data/price"
URL_MINUTE_OHLCV = "https://min-api.cryptocompare.com/data/v2/histominute"

HEADERS = {
        "authorization": "Apikey " + CC_SECRET_KEY
    }

This is the main block to provide a solution for Q1. The function gbtc_discount_ratio() calculates the discount-to-NAV ratio for GBTC, which is the percentage difference between the theoretical value of the trust fund and it's observed market price. When the ratio is negative, fund is trading lower than its theoretical value, and hence at a discount(cheap), and vis-versa, at a premium if the ratio is positive.

To achieve this, we need the CNBC quote for the observed market price and the price of BTCUSD to calculate the GBTC's NAV (which is BTC per share * BTC Price). We will use BeautifulSoup and the CC API for market price of GBTC and the spot price of BTC, respectively.

There are two portions then which are commented below: the **BeautifulSoup** portion and the **CC API** portion. 

The BeautifulSoup portion basically gets all the HTML code located on the CNBC page that provides quotes for GBTC. It then grabs the quote from the HTML and converts it to float number to make it readily available for the discount-to-NAV calculation.

The CC API portion is the main calculation and contributes the final proposed solution. 

In [ ]:
def gbtc_discount_ratio():
    #BeautifulSoup portion
    html = requests.get("https://www.cnbc.com/quotes/GBTC").content
    soup = BeautifulSoup(html, 'lxml')
    gbtc_price_container = soup.find('span', attrs = {'class':"QuoteStrip-lastPrice"})
    market_price = float(gbtc_price_container.text)
    print("GBTC is currently trading at " + str(market_price))
    
    #CC API portion
    btc_per_share = 0.00092683
    payload = {
        "fsym": "BTC",
        "tsyms": QUOTE_CURRENCY
    }
    btc_price = requests.get(URL_SINGLE_PRICE, headers = HEADERS, params=payload).json()
    #Discount-to-NAV Equation
    discount = 100 * ((market_price / (btc_per_share * btc_price["USD"])) - 1)
    print("The discount ratio for GBTC is currently at " + str(round(discount, 3)) + "%")
    return

gbtc_discount_ratio()




### Question 2

Please write a script that will trigger price alert messages according to the below parameters, using
historical data from the CryptoCompare API Your script should be able to work for any commonly
used crypto assets (e.g. BTC, ETH, SOL) listed on CryptoCompare.


+/- 1.00% over 2 minutes
<br>+/- 2.00% over 5 minutes
<br>+/- 3.00% over 15 minutes
<br>+/- 4.00% over 90 minutes
<br>+/- 5.00% over 12 hours

#### Solution

We have three functions necessary for this solution:

* **minute_price_hist()** - This function generates the data necessary for the percentage change calculations. We send the payload in the JSON structure as required by CC. Below are the following parameter definitions.
> **symbol** - Accepts a valid coin ticker symbol in string format <br> **limit**  - Passes the amount of requested  rows of available data for the symbol of interest. As previously mentioned, 2,000 is the maximum limit.<br> **aggregate** - Set to 1. This is because we need minute-by-minute data. There is a range of valid values, up to 30. If we set it 30, we will get data points in 30 minute increments.<br> **exchange** - Set to ''. Uses CC API proprietary price aggregation model to generate a price for the symbol of interest. You can change it to a specific exchange if one wishes to.

* **percent_change** - This function performs the percent change calculation between a new and old close
> **delta** - Specifies the desired amount of unit difference in time between two closes

* **alerts** - The function performs the necessary conditional comparisons across different time frames as required. To understand this function, let's walk through an example. If we pass 2 as an argument for **alerts()**, it is passed into **mod_mins** (*mod_mins takes care of the fact that when performing percentage change on a list, the first entry is the 0th entry. If we did not offset by one, we would be performing minute-by-minute % change in this example*). The variable **mod_mins** is passed into the **percent_change()** function, where the rate of change is stored for later comparision for the appropriate conditional if statements as the **pct_chg** variable.
> **minutes** - Takes an integer value of minutes that you would like the function to run through the condtional if engine

In [ ]:
def minute_price_hist(symbol, limit, aggregate = 1, exchange = ''): 
    payload = {
        "fsym": symbol,
        "tsym": QUOTE_CURRENCY,
        "limit": limit,
        "aggregate": aggregate,
        "e:": exchange
        }

    result = requests.get(URL_MINUTE_OHLCV, headers = HEADERS, params = payload).json()
    data = pd.DataFrame(result["Data"]["Data"])
   
    return data

def percent_change(delta):
    new = data["close"].iloc[-1]
    old = data["close"].iloc[-delta]

    return (((new - old) / old) * 100)

def alerts(minutes):
    #See the Jupyter notebook for notes on this variable
    mod_mins = minutes + 1
    
    pct_chg = percent_change(mod_mins)
    #The assignment asked for certain absolute value rates of change across different timeframes,
    #but you may change them to custom values by changing the list below.
    
    #If one wishes to see if the alerts() function works properly, set all of the values in the list to 0
    #abs_chg = [0,0,0,0,0]
    abs_chg = [1,2,3,4,5]
    
    #Grabs the current time
    now = datetime.datetime.now()
    
    #Conditional If Engine. As minutes and absolute percent change conditions are met, statements are printed.
    if minutes >= 2 and minutes <= 4:
        if abs(pct_chg) >= abs_chg[0]:
            print(now.strftime("%H:%M:%S") + " The price of " + SYMBOL + "/" + QUOTE_CURRENCY +  " has changed by " + str(round((pct_chg), 2)) + \
                  "% over the past " + str(minutes) + " minutes")
    elif minutes >= 5 and minutes <= 14:
        if abs(pct_chg) >= abs_chg[1]:
            print(now.strftime("%H:%M:%S") + " The price of " + SYMBOL + "/" + QUOTE_CURRENCY +  " has changed by " + str(round((pct_chg), 2)) + \
                  "% over the past " + str(minutes) + " minutes")    
    elif minutes >= 15 and minutes <= 89:
        if abs(pct_chg) >= abs_chg[2]:
            print(now.strftime("%H:%M:%S") + " The price of " + SYMBOL + "/" + QUOTE_CURRENCY +  " has changed by " + str(round((pct_chg), 2)) + \
                  "% over the past " + str(minutes) + " minutes")
    elif minutes >= 90 and minutes <= 719:
        if abs(pct_chg) >= abs_chg[3]:
            print(now.strftime("%H:%M:%S") + " The price of " + SYMBOL + "/" + QUOTE_CURRENCY +  " has changed by " + str(round((pct_chg), 2)) + \
                  "% over the past " + str(minutes / 60) + " hours")
    elif minutes >= 720:
        if abs(pct_chg) >= abs_chg[4]:
            print(now.strftime("%H:%M:%S") + " The price of " + SYMBOL + "/" + QUOTE_CURRENCY + " has changed by " + str(round((pct_chg), 2)) + \
                  "% over the past " + str(minutes / 60) + " hours")
    return

The main block that provides a solution for Q2. The assignment requested for alerts at 2, 5, 15, 90, and 720 (12 hours) minutes. However, you may change it to custom timeframes. However, none of the value may exceed **MAX_MINS** or the code will break. The loop_rate sets how quickly the infinite while loop will carry out. In this case, loop_rate is set to 5 seconds.

The simplified loop flow goes as follows:
1. Grab the minute historical data for the symbol of interest<br>
2. Call the alerts() function at the 5 desired alter times, which prints whatever required conditions are met<br>
3. Pause the loop for the desired amount of time
4. Repeat

In [ ]:
alert_times = [2,5,15,90,720]
loop_rate  = 5

while True:    
    data = minute_price_hist(SYMBOL, MAX_MINS)
    
    alerts(alert_times[0])
    alerts(alert_times[1])
    alerts(alert_times[2])
    alerts(alert_times[3])
    alerts(alert_times[4])

    print("------------------------")    
    time.sleep(loop_rate)